# Pipeline Overview

#### 1. Use CLIP to Generate Tags:

Use CLIP to generate descriptive tags for your moodboard images.

#### 2.Search for Similar Images Using Yandex:

Use the generated tags to perform a reverse image search on Yandex.

#### 3.Build a Basic Interface:

Create a Python interface where you can right-click on images to search for similar ones.

### Step 1: Use CLIP to Generate Tags

In [ ]:
%pip install torch torchvision transformers pillow requests

### Python Code for CLIP Tag Generation

In [ ]:
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel

# Load pre-trained CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Load your moodboard image
image = Image.open("moodboard_image.jpg")

# Define possible tags (you can customize this list)
tags = ["fantasy", "cyberpunk", "landscape", "portrait", "anime", "realistic", "dark", "bright", "minimalist", "detailed"]

# Process the image and tags
inputs = processor(text=tags, images=image, return_tensors="pt", padding=True)

# Get CLIP embeddings
with torch.no_grad():
    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image  # Image-text similarity scores
    probs = logits_per_image.softmax(dim=1)     # Convert to probabilities

# Get top tags
top_k = 5  # Number of top tags to retrieve
top_probs, top_indices = probs.topk(top_k, dim=1)

# Print top tags
for i in range(top_k):
    print(f"Tag: {tags[top_indices[0][i]]}, Probability: {top_probs[0][i].item():.4f}")

### Step 2: Search for Similar Images Using Yandex
To automate Yandex reverse image search, you can use the requests library to upload the image and scrape the results.

In [ ]:
import requests
from bs4 import BeautifulSoup

def yandex_reverse_image_search(image_path):
    # Yandex reverse image search URL
    url = "https://yandex.com/images/search"

    # Open the image file
    with open(image_path, "rb") as image_file:
        # Upload the image to Yandex
        response = requests.post(url, files={"upfile": image_file})

    # Parse the response (this is a simplified example)
    soup = BeautifulSoup(response.text, "html.parser")
    results = soup.find_all("img", class_="serp-item__thumb")  # Adjust based on Yandex's HTML structure

    # Extract image URLs
    similar_images = [img["src"] for img in results]
    return similar_images

# Example usage
image_path = "moodboard_image.jpg"
similar_images = yandex_reverse_image_search(image_path)
print("Similar Images:", similar_images)

### Step 3: Build a Basic Interface
You can use tkinter to create a simple interface where you can right-click on images to search for similar ones.

In [ ]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import requests
from io import BytesIO

# Function to handle right-click event
def search_similar_images(event):
    # Get the image path
    image_path = "moodboard_image.jpg"  # Replace with your image path

    # Perform Yandex reverse image search
    similar_images = yandex_reverse_image_search(image_path)

    # Display the first similar image (for demonstration)
    if similar_images:
        response = requests.get(similar_images[0])
        img = Image.open(BytesIO(response.content))
        img = img.resize((200, 200), Image.ANTIALIAS)
        img_tk = ImageTk.PhotoImage(img)
        similar_image_label.config(image=img_tk)
        similar_image_label.image = img_tk

# Create the main window
root = tk.Tk()
root.title("Moodboard Image Search")

# Load and display the moodboard image
image = Image.open("moodboard_image.jpg")
image = image.resize((400, 400), Image.ANTIALIAS)
image_tk = ImageTk.PhotoImage(image)
image_label = tk.Label(root, image=image_tk)
image_label.pack()

# Bind right-click event to the image
image_label.bind("<Button-3>", search_similar_images)

# Label to display similar images
similar_image_label = tk.Label(root)
similar_image_label.pack()

# Run the application
root.mainloop()

## How It Works

### CLIP Tag Generation:
CLIP generates descriptive tags for your moodboard image.

### Yandex Reverse Image Search:
The image is uploaded to Yandex, and similar images are retrieved.

<h1>Basic Interface:</h1>
<ul><li>A tkinter window displays the moodboard image.</li>
<li>Right-clicking on the image triggers a Yandex reverse image search.</li>
<li>The first similar image is displayed in the interface.</li></ul>

## How It Works

### Improve Yandex Scraping:
CLIP generates descriptive tags for your moodboard image.

### Enhance the Interface:
The image is uploaded to Yandex, and similar images are retrieved.

### Optimize CLIP:
Fine-tune CLIP on a custom dataset for better tag generation.
